In [ ]:
import tensorflow as tf
import pandas as pd
from pprint import pprint
import numpy as np
import tensorflow_reccomenders as tfrs
import tensorflow_datasets as tfds

ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

In [ ]:
ratings = rating.map(lambda x : {
    "user_id" : x["user_id"],
    "movie_title" : x["movie_title"],
    "timestamp" : x["timestamp"]
})

moives = movies.map(lambda x: x["movie_title"])